In [4]:
import csv
import math
def areawise(freqfile,percentfile,P,A,K):
    csvfile = open(percentfile, newline='')
    csvAvg = open(freqfile,newline='')
    
    c = csv.reader(csvfile)
    d = csv.reader(csvAvg)
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    average=[0] * 71  # change to higher value to make it non-redundant for now it is equal to the no of total unique crimes
    i=0
    yearsToTrain = 14    # change s value to no of years to be used for training 
    
    for row in d:
        if row[0] == 'TOTAL'or row[0]=='Category' or row[0]=='year' or row[0]=='month':
            continue
        else:
           j=0
           for month in range (1,(yearsToTrain*12)+1):
               if float(row[month]) !=0:    
                   j+=1
                   average[i] += float(row[month])    # maintain grand average for scale later 
           if average[i] != 0:
                average[i] /= j
           i+=1
          
    percentages = []
    totals = []
    categories =[]
    i = 0
    monYear = []
    
    for row in c:
        
        if row[0] == 'TOTAL'or row[0]=='Category' or row[0]=='year':
            continue
        elif row[0]=='month':
            for month in range(1,(yearsToTrain*12)+1):
                monYear.append(float(row[month]))
        else:
           temp = []
           categories.append(row[0])
           
           for month in range (1,(yearsToTrain*12)+1):
               if float(row[month]) !=0:    
                   j+=1
               
               temp.append(float(row[month]))   # add monthly percentages of a particular crime to  list 
           
           percentages.append(temp)   # add monthly percentages list to a list 
           i += 1
    
    # print("percentages here :")
    # print(percentages)
    predictions=[]


    def predictOverYears(start,end):           # fn to give prediction for one year mentioned by start and end
        predictAyear = []
        for idx in range (0,i):
            crime = percentages[idx]
            sum =0
            index = -1
            for it in range (start,end):     # sum for caluclationg confidence later on             
                sum+=crime[it]
        
            predictMonth = []
    
            for it in range (start,end):
                avg = sum - crime[it]
                avg=float(avg/11)
                if avg != 0:
                    likeness = ((float(crime[it])/(avg))) * 100    # measure confidence rate for that month for that crime
                else :
                    likeness=0
                temp = [likeness,monYear[it]]
                predictMonth.append(temp)                   # append confidence rate and index of month predicted
    
            predictAyear.append(predictMonth)           # append all month's of that year likliness to a list 
        return predictAyear


    for j in range(0,yearsToTrain):            
        n=j*12
        predictions.append(predictOverYears(n,n+12))
    
    # print(predictions[1])
    # print("predictions here :")
    # print(predictions)
    
    predictionPerCrime = []
    for idx in range(0,i):
        tem = []
        for k in range(0,yearsToTrain):
            tem.append(predictions[k][idx])           # get each individual year predictions and add it to a new list 
        predictionPerCrime.append(tem)
        
    
    # print("prediction per crime:")
    # print(predictionPerCrime[0])


    grandPrediction = []
    for idx in range(0,i):
        perCrimeConf=[]
        for month in range(0,12):
            sumOverYears=0
            for yr in range(0,yearsToTrain):
                sumOverYears+=predictionPerCrime[idx][yr][month][0]
            tem=(  sumOverYears /  yearsToTrain )  # sum of confidence of a month across years and divide by yearsToTrain
            perCrimeConf.append(tem)
        grandPrediction.append(perCrimeConf)  # append confidence rates of each month per crime to list 

    
    
    # print(grandPrediction[0][2])
    
    for j in range(0,i):
        low=9999
        for m in range(0,12):
            if grandPrediction[j][m] < low:
                low = grandPrediction[j][m]
        if low != 0 :
             low = math.ceil(low) -2  # find lowest value ceil it and subtrat 2 from it
        for n in range(0,12):
            grandPrediction[j][n] -= low  # then subtract this val(low) from all
            grandPrediction[j][n] *= (K/(math.sqrt(P)*A))
    
    # print("grand predictions: ")
    # print(grandPrediction)
    
    # for j in range(0,i):
    #     for k in range(0,12):
    #         grandPrediction[j][k] *= (math.sqrt(P)/A)

    print("{")
    print('\"'+freqfile.split('_')[0]+'\":')
    print("[")
    
    for j in range(0,6):
        print('{')
        print('\"'+"crime"+'\":'+'\"'+categories[j]+'\",')
        max = 0
        avg = 0
        for m in range (0,12):
             avg+=grandPrediction[j][m]
             if grandPrediction[j][m] > max :
                 max = grandPrediction[j][m]
        avg=avg/12
        ideal = ( max + avg)/2     # compute ideal confidence and acceptable confidence level
        print('\"'+"overallAverage"+'\":'+'\"'+str(average[j])+'\",')
        print('\"'+"idealConfidence"+'\":'+'\"'+str(ideal)+'\",')
        print('\"'+"accepatableConfidence"+'\":'+'\"'+str(avg)+'\",')
        print('\"'+"confidenceRates"+'\":'+'[')
        print('{')
        for k in range(0,12):
            if k != 11:
                print('\"'+months[k]+'\":'+'\"'+str(grandPrediction[j][k])+'\",')
            else:
                print('\"'+months[k]+'\":'+'\"'+str(grandPrediction[j][k])+'\"')
        # print(j+1,")",categories[j],"With overall average:",average[j])
        # print("With ideal confidence:",ideal,"and acceptable confidence:",avg,"monthly predictons are:")
        # for k in range(0,12):
        #     print("     Month",k+1,"confidence rate:",grandPrediction[j][k]) 
        print('}')  
        print(']')
        if j!=i-1:
            print('},')
        else:
            print('}')
                 
    print(']')
    print('}')


areawise('BRONX_Frequency.csv','BRONX_PercentageTotal.csv',1472654,109.3,1000000)
areawise('BROOKLYN_Frequency.csv','BROOKLYN_PercentageTotal.csv',2736074,179.7,1000000)
areawise('MANHATTAN_Frequency.csv','MANHATTAN_PercentageTotal.csv',1694251,58.8,1000000)
areawise('QUEENS_Frequency.csv','QUEENS_PercentageTotal.csv',2405464,281.5,1000000)
areawise('STATEN_ISLAND_Frequency.csv','STATEN_ISLAND_PercentageTotal.csv',495747,148.9,1000000)




{
"BRONX":
[
{
"crime":"HARRASSMENT 2",
"overallAverage":"1145.9345238095239",
"idealConfidence":"67.0677113314428",
"accepatableConfidence":"52.96785791990526",
"confidenceRates":[
{
"Jan":"39.027090285998426",
"Feb":"25.686588604815096",
"Mar":"49.74836487428922",
"Apr":"59.48668262299869",
"May":"78.0518729147385",
"Jun":"78.86236663526898",
"Jul":"45.55043498030579",
"Aug":"12.225566580142676",
"Sep":"81.16756474298033",
"Oct":"66.66069042406627",
"Nov":"45.073971946141924",
"Dec":"54.07310042711717"
}
]
},
{
"crime":"PETIT LARCENY",
"overallAverage":"1191.125",
"idealConfidence":"107.22974886903219",
"accepatableConfidence":"83.22899640434504",
"confidenceRates":[
{
"Jan":"68.58055672861323",
"Feb":"13.728157546185843",
"Mar":"25.370666309330154",
"Apr":"52.00243672145583",
"May":"75.73367949346601",
"Jun":"97.25774010572805",
"Jul":"94.9133220248541",
"Aug":"111.33175587001206",
"Sep":"114.32882966905028",
"Oct":"110.7780284984663",
"Nov":"103.49228255125949",
"Dec":"131.23050133